In [1]:
!pip install pandas
!pip install scikit-surprise


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 4.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3227955 sha256=22b7737073a689ddb2a29bc283e897f57f94b10641199caf9a4e7115a9881bc1
  Stored in directory: /home/codespace/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from collections import defaultdict
from surprise import KNNWithMeans
from surprise import SVD, SVDpp
from surprise import KNNBaseline
from surprise import KNNBasic
from surprise import KNNWithZScore
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
from surprise.model_selection import KFold
from surprise.model_selection import GridSearchCV

import time

In [3]:
start_time = time.time()
df = pd.read_csv("OrderHistory.csv")
df.head()
computational_time = time.time() - start_time
print('Done in %0.3fs' %(computational_time))

Done in 0.384s


In [4]:
df

,Unnamed: 0,ProductId,UserId,Score,Age,Gender,State,Product_name,Brand
0,1,1,A1D87F6ZCVE5NK,1,77,0,Bihar,Alisha Solid Women's Cycling Shorts,Alisha
1,2,2,ABXLMWJIXXAIN,4,75,0,Tamil Nadu,FabHomeDecor Fabric Double Sofa Bed,FabHomeDecor
2,3,3,A395BORC6FGVXV,2,22,1,Himachal Pradesh,AW Bellies,AW
3,4,4,A1UQRSCLF8GW1T,5,12,0,Meghalaya,Alisha Solid Women's Cycling Shorts,Alisha
4,5,4,ADT0SRK1MGOEU,4,33,0,Madhya Pradesh,Sicons All Purpose Arnica Dog Shampoo,Sicons
...,...,...,...,...,...,...,...,...,...
229994,229995,29370,A2MKDOEB6K6KS1,5,16,0,Haryana,Starsy Printed Women's Round Neck White T-Shirt,Starsy
229995,229996,29370,A3DZMDTBJ7PQX7,5,19,1,Himachal Pradesh,Starsy Printed Women's Round Neck White T-Shirt,Starsy
229996,229997,29370,A2TNGO6SGRH8A7,5,79,0,Telangana,Starsy Printed Women's Round Neck White T-Shirt,Starsy
229997,229998,29370,A1YFK6EHSVJKNH,5,51,0,Punjab,Starsy Printed Women's Round Neck White T-Shirt,Starsy


In [5]:
df.dtypes

Unnamed: 0       int64
ProductId        int64
UserId          object
Score            int64
Age              int64
Gender           int64
State           object
Product_name    object
Brand           object
dtype: object

In [6]:
rows_count, columns_count = df.shape
print('Total Number of rows :', rows_count)
print('Total Number of columns :', columns_count)

Total Number of rows : 229999
Total Number of columns : 9


In [7]:
df.dtypes

Unnamed: 0       int64
ProductId        int64
UserId          object
Score            int64
Age              int64
Gender           int64
State           object
Product_name    object
Brand           object
dtype: object

In [8]:
unique_userId = df['UserId'].nunique()
unique_productId = df['ProductId'].nunique()
print('Total number of unique Users    : ', unique_userId)
print('Total number of unique Products : ', unique_productId)

Total number of unique Users    :  135068
Total number of unique Products :  29370


In [9]:
df1 = df.dropna()

In [10]:
df1.apply(lambda x : sum(x.isnull()))

Unnamed: 0      0
ProductId       0
UserId          0
Score           0
Age             0
Gender          0
State           0
Product_name    0
Brand           0
dtype: int64

In [11]:
df1

,Unnamed: 0,ProductId,UserId,Score,Age,Gender,State,Product_name,Brand
0,1,1,A1D87F6ZCVE5NK,1,77,0,Bihar,Alisha Solid Women's Cycling Shorts,Alisha
1,2,2,ABXLMWJIXXAIN,4,75,0,Tamil Nadu,FabHomeDecor Fabric Double Sofa Bed,FabHomeDecor
2,3,3,A395BORC6FGVXV,2,22,1,Himachal Pradesh,AW Bellies,AW
3,4,4,A1UQRSCLF8GW1T,5,12,0,Meghalaya,Alisha Solid Women's Cycling Shorts,Alisha
4,5,4,ADT0SRK1MGOEU,4,33,0,Madhya Pradesh,Sicons All Purpose Arnica Dog Shampoo,Sicons
...,...,...,...,...,...,...,...,...,...
229994,229995,29370,A2MKDOEB6K6KS1,5,16,0,Haryana,Starsy Printed Women's Round Neck White T-Shirt,Starsy
229995,229996,29370,A3DZMDTBJ7PQX7,5,19,1,Himachal Pradesh,Starsy Printed Women's Round Neck White T-Shirt,Starsy
229996,229997,29370,A2TNGO6SGRH8A7,5,79,0,Telangana,Starsy Printed Women's Round Neck White T-Shirt,Starsy
229997,229998,29370,A1YFK6EHSVJKNH,5,51,0,Punjab,Starsy Printed Women's Round Neck White T-Shirt,Starsy


In [12]:
df_transpose = df1.describe().T
df_transpose

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,166691.0,113800.004361,67310.789438,1.0,52237.5,115063.0,174165.5,229999.0
ProductId,166691.0,14465.129797,8616.836864,1.0,6521.0,14582.0,22350.0,29370.0
Score,166691.0,4.173891,1.312486,1.0,4.0,5.0,5.0,5.0
Age,166691.0,43.977341,21.130598,8.0,26.0,44.0,62.0,80.0
Gender,166691.0,0.499685,0.500001,0.0,0.0,0.0,1.0,1.0


In [13]:
df1['Score'].value_counts()

Score
5    105615
4     24044
1     15373
3     12808
2      8851
Name: count, dtype: int64

In [14]:
def map_score(score):
    if isinstance(score, str) and score.lower() == 'score':
        return score
    return int(float(score))
df1['Mapped_Score'] = df1['Score'].map(map_score)


/tmp/ipykernel_5487/1642850552.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Mapped_Score'] = df1['Score'].map(map_score)


In [15]:
df1['Mapped_Score'].value_counts()


Mapped_Score
5    105615
4     24044
1     15373
3     12808
2      8851
Name: count, dtype: int64

In [16]:
df1

,Unnamed: 0,ProductId,UserId,Score,Age,Gender,State,Product_name,Brand,Mapped_Score
0,1,1,A1D87F6ZCVE5NK,1,77,0,Bihar,Alisha Solid Women's Cycling Shorts,Alisha,1
1,2,2,ABXLMWJIXXAIN,4,75,0,Tamil Nadu,FabHomeDecor Fabric Double Sofa Bed,FabHomeDecor,4
2,3,3,A395BORC6FGVXV,2,22,1,Himachal Pradesh,AW Bellies,AW,2
3,4,4,A1UQRSCLF8GW1T,5,12,0,Meghalaya,Alisha Solid Women's Cycling Shorts,Alisha,5
4,5,4,ADT0SRK1MGOEU,4,33,0,Madhya Pradesh,Sicons All Purpose Arnica Dog Shampoo,Sicons,4
...,...,...,...,...,...,...,...,...,...,...
229994,229995,29370,A2MKDOEB6K6KS1,5,16,0,Haryana,Starsy Printed Women's Round Neck White T-Shirt,Starsy,5
229995,229996,29370,A3DZMDTBJ7PQX7,5,19,1,Himachal Pradesh,Starsy Printed Women's Round Neck White T-Shirt,Starsy,5
229996,229997,29370,A2TNGO6SGRH8A7,5,79,0,Telangana,Starsy Printed Women's Round Neck White T-Shirt,Starsy,5
229997,229998,29370,A1YFK6EHSVJKNH,5,51,0,Punjab,Starsy Printed Women's Round Neck White T-Shirt,Starsy,5


In [17]:
rating_counts = pd.DataFrame(df1['Mapped_Score'].value_counts()).reset_index()
rating_counts.columns = ['Labels', 'Ratings']
rating_counts

,Labels,Ratings
0,5,105615
1,4,24044
2,1,15373
3,3,12808
4,2,8851


In [18]:
df1

,Unnamed: 0,ProductId,UserId,Score,Age,Gender,State,Product_name,Brand,Mapped_Score
0,1,1,A1D87F6ZCVE5NK,1,77,0,Bihar,Alisha Solid Women's Cycling Shorts,Alisha,1
1,2,2,ABXLMWJIXXAIN,4,75,0,Tamil Nadu,FabHomeDecor Fabric Double Sofa Bed,FabHomeDecor,4
2,3,3,A395BORC6FGVXV,2,22,1,Himachal Pradesh,AW Bellies,AW,2
3,4,4,A1UQRSCLF8GW1T,5,12,0,Meghalaya,Alisha Solid Women's Cycling Shorts,Alisha,5
4,5,4,ADT0SRK1MGOEU,4,33,0,Madhya Pradesh,Sicons All Purpose Arnica Dog Shampoo,Sicons,4
...,...,...,...,...,...,...,...,...,...,...
229994,229995,29370,A2MKDOEB6K6KS1,5,16,0,Haryana,Starsy Printed Women's Round Neck White T-Shirt,Starsy,5
229995,229996,29370,A3DZMDTBJ7PQX7,5,19,1,Himachal Pradesh,Starsy Printed Women's Round Neck White T-Shirt,Starsy,5
229996,229997,29370,A2TNGO6SGRH8A7,5,79,0,Telangana,Starsy Printed Women's Round Neck White T-Shirt,Starsy,5
229997,229998,29370,A1YFK6EHSVJKNH,5,51,0,Punjab,Starsy Printed Women's Round Neck White T-Shirt,Starsy,5


In [19]:
df1 = df1.drop(df1[df1['Score'] == 'Score'].index)
df1['Mapped_Score'].value_counts()


Mapped_Score
5    105615
4     24044
1     15373
3     12808
2      8851
Name: count, dtype: int64

In [20]:
rating_counts = pd.DataFrame(df1['Mapped_Score'].value_counts()).reset_index()
rating_counts.columns = ['Labels', 'Ratings']
rating_counts

,Labels,Ratings
0,5,105615
1,4,24044
2,1,15373
3,3,12808
4,2,8851


In [21]:
df1.corr()

ValueError: could not convert string to float: 'A1D87F6ZCVE5NK'

In [22]:
df1.head()

,Unnamed: 0,ProductId,UserId,Score,Age,Gender,State,Product_name,Brand,Mapped_Score
0,1,1,A1D87F6ZCVE5NK,1,77,0,Bihar,Alisha Solid Women's Cycling Shorts,Alisha,1
1,2,2,ABXLMWJIXXAIN,4,75,0,Tamil Nadu,FabHomeDecor Fabric Double Sofa Bed,FabHomeDecor,4
2,3,3,A395BORC6FGVXV,2,22,1,Himachal Pradesh,AW Bellies,AW,2
3,4,4,A1UQRSCLF8GW1T,5,12,0,Meghalaya,Alisha Solid Women's Cycling Shorts,Alisha,5
4,5,4,ADT0SRK1MGOEU,4,33,0,Madhya Pradesh,Sicons All Purpose Arnica Dog Shampoo,Sicons,4


In [24]:
df1


,id,ProductId,Brand,Color,ProductName,UserId,Score,Age,Gender,State,Mapped_Score
1,1.0,B00813GRG4,Gucci,brown,clock,A1D87F6ZCVE5NK,1,77,0,Bihar,1
2,2.0,B000LQOCH0,Samsung,pink,jacket,ABXLMWJIXXAIN,4,75,0,Tamil Nadu,4
3,3.0,B000UA0QIQ,Prada,blue,jacket,A395BORC6FGVXV,2,22,1,Himachal Pradesh,2
4,4.0,B006K2ZZ7K,Honda,grey,laptopsleeve,A1UQRSCLF8GW1T,5,12,0,Meghalaya,5
5,5.0,B006K2ZZ7K,Honda,grey,laptopsleeve,ADT0SRK1MGOEU,4,33,0,Madhya Pradesh,4
...,...,...,...,...,...,...,...,...,...,...,...
299996,299996.0,B000VA5LZI,Apple,grey,pen,AE6YV1KF6UKJR,2.0,63.0,0.0,Madhya Pradesh,2
299997,299997.0,B000VA5LZI,Apple,grey,pen,A3NRNASNTU5TFI,5.0,29.0,0.0,Uttarakhand,5
299998,299998.0,B0007XAUXC,H&M,pink,diary,AYRFQU12QD60L,5.0,35.0,0.0,Gujarat,5
299999,299999.0,B0007XAUXC,H&M,pink,diary,A1TDZMXV5P6AJB,4.0,53.0,0.0,Telangana,4


In [23]:
df1 = df1.drop(['HelpfulnessNumerator'], axis=1)
df1 = df1.drop(['HelpfulnessDenominator'], axis=1)
df1 = df1.drop(['Summary'],axis = 1)
df1 = df1.drop(['Text'],axis = 1)
df1 = df1.drop(['Score'],axis = 1)

KeyError: "['HelpfulnessNumerator'] not found in axis"

In [24]:
df1.head()


,Unnamed: 0,ProductId,UserId,Score,Age,Gender,State,Product_name,Brand,Mapped_Score
0,1,1,A1D87F6ZCVE5NK,1,77,0,Bihar,Alisha Solid Women's Cycling Shorts,Alisha,1
1,2,2,ABXLMWJIXXAIN,4,75,0,Tamil Nadu,FabHomeDecor Fabric Double Sofa Bed,FabHomeDecor,4
2,3,3,A395BORC6FGVXV,2,22,1,Himachal Pradesh,AW Bellies,AW,2
3,4,4,A1UQRSCLF8GW1T,5,12,0,Meghalaya,Alisha Solid Women's Cycling Shorts,Alisha,5
4,5,4,ADT0SRK1MGOEU,4,33,0,Madhya Pradesh,Sicons All Purpose Arnica Dog Shampoo,Sicons,4


In [25]:
users_counts = df1['UserId'].value_counts().rename('users_counts')
users_data   = df1.merge(users_counts.to_frame(),
                                left_on='UserId',
                                right_index=True)

In [26]:
subset_df = users_data[users_data.users_counts>=3]
subset_df

,Unnamed: 0,ProductId,UserId,Score,Age,Gender,State,Product_name,Brand,Mapped_Score,users_counts
9,10,7,A3HDKO7OW0QNK4,5,60,1,Madhya Pradesh,Alisha Solid Women's Cycling Shorts,Alisha,5,3
28,29,11,A3HDKO7OW0QNK4,5,60,1,Madhya Pradesh,Ladela Bellies,Ladela,5,3
195370,195371,24936,A3HDKO7OW0QNK4,5,60,1,Madhya Pradesh,Twella Creations Evergreen Glass Necklace,Twella Creations,5,3
12,13,9,A18ECVX2RJ7HUE,4,36,1,Arunachal Pradesh,"dilli bazaaar Bellies, Corporate Casuals, Casuals",dilli bazaaar,4,7
33369,33370,4035,A18ECVX2RJ7HUE,3,36,1,Arunachal Pradesh,"UpTown Metal, Alloy Necklace",UpTown,3,7
...,...,...,...,...,...,...,...,...,...,...,...
217259,217260,27680,A9MJ0YE1HFQF5,5,30,0,Andhra Pradesh,Affinity Jewellers Just like diamonds Cubic Zi...,Affinity Jewellers,5,3
218888,218889,27898,A9MJ0YE1HFQF5,5,30,0,Andhra Pradesh,Ace HEADPHONE151ONTO Stereo Dynamic Headphone ...,Ace,5,3
220496,220497,28094,A1KDIFVBAT4MWE,3,55,0,Tripura,Vetra E00TY112 Multifunction Car Tray For Maru...,Vetra,3,3
220497,220498,28094,A1KDIFVBAT4MWE,3,55,0,Tripura,Vetra E00TY112 Multifunction Car Tray For Maru...,Vetra,3,3


In [28]:
product_rating_counts = subset_df['ProductId'].value_counts().rename('product_rating_counts')
product_rating_data   = subset_df.merge(product_rating_counts.to_frame(),
                                left_on='ProductId',
                                right_index=True)
product_rating_data = product_rating_data[product_rating_data.product_rating_counts >= 5]
product_rating_data

,Unnamed: 0,ProductId,UserId,Score,Age,Gender,State,Product_name,Brand,Mapped_Score,users_counts,product_rating_counts
12,13,9,A18ECVX2RJ7HUE,4,36,1,Arunachal Pradesh,"dilli bazaaar Bellies, Corporate Casuals, Casuals",dilli bazaaar,4,7,5
13,14,9,A2MUGFV2TDQ47K,5,60,0,Manipur,Alisha Solid Women's Cycling Shorts,Alisha,5,49,5
17,18,9,A2A9X58G2GTBLP,5,68,1,Chhattisgarh,Alisha Solid Women's Cycling Shorts,Alisha,5,16,5
22,23,9,AJ613OLZZUG7V,5,47,0,Rajasthan,Alisha Solid Women's Cycling Shorts,Alisha,5,3,5
24,25,9,A3FONPR03H3PJS,5,56,0,Chhattisgarh,Alisha Solid Women's Cycling Shorts,Alisha,5,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...
173337,173338,22245,A16H248EAOBV6U,5,15,1,Jharkhand,PrettySecrets Fashion Women's Push-up Bra,Brown,5,4,5
173338,173339,22245,A3BGO7U46LWLUR,5,29,1,Punjab,Madaam NonPadded Women's Tube Bra,Multicolor,5,4,5
173339,173340,22245,A2SCOK1P7Q9N9L,5,75,1,Andhra Pradesh,Madaam NonPadded Women's Tube Bra,Multicolor,5,4,5
173340,173341,22245,A116SXAVY89LJI,5,14,0,Gujarat,Madaam NonPadded Women's Tube Bra,Multicolor,5,4,5


In [29]:
product_rating_data

,Unnamed: 0,ProductId,UserId,Score,Age,Gender,State,Product_name,Brand,Mapped_Score,users_counts,product_rating_counts
12,13,9,A18ECVX2RJ7HUE,4,36,1,Arunachal Pradesh,"dilli bazaaar Bellies, Corporate Casuals, Casuals",dilli bazaaar,4,7,5
13,14,9,A2MUGFV2TDQ47K,5,60,0,Manipur,Alisha Solid Women's Cycling Shorts,Alisha,5,49,5
17,18,9,A2A9X58G2GTBLP,5,68,1,Chhattisgarh,Alisha Solid Women's Cycling Shorts,Alisha,5,16,5
22,23,9,AJ613OLZZUG7V,5,47,0,Rajasthan,Alisha Solid Women's Cycling Shorts,Alisha,5,3,5
24,25,9,A3FONPR03H3PJS,5,56,0,Chhattisgarh,Alisha Solid Women's Cycling Shorts,Alisha,5,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...
173337,173338,22245,A16H248EAOBV6U,5,15,1,Jharkhand,PrettySecrets Fashion Women's Push-up Bra,Brown,5,4,5
173338,173339,22245,A3BGO7U46LWLUR,5,29,1,Punjab,Madaam NonPadded Women's Tube Bra,Multicolor,5,4,5
173339,173340,22245,A2SCOK1P7Q9N9L,5,75,1,Andhra Pradesh,Madaam NonPadded Women's Tube Bra,Multicolor,5,4,5
173340,173341,22245,A116SXAVY89LJI,5,14,0,Gujarat,Madaam NonPadded Women's Tube Bra,Multicolor,5,4,5


In [30]:
amazon_df = product_rating_data.copy()

In [31]:
panda_data = amazon_df.drop(['users_counts', 'product_rating_counts'], axis=1)

In [32]:
panda_data.head()

,Unnamed: 0,ProductId,UserId,Score,Age,Gender,State,Product_name,Brand,Mapped_Score
12,13,9,A18ECVX2RJ7HUE,4,36,1,Arunachal Pradesh,"dilli bazaaar Bellies, Corporate Casuals, Casuals",dilli bazaaar,4
13,14,9,A2MUGFV2TDQ47K,5,60,0,Manipur,Alisha Solid Women's Cycling Shorts,Alisha,5
17,18,9,A2A9X58G2GTBLP,5,68,1,Chhattisgarh,Alisha Solid Women's Cycling Shorts,Alisha,5
22,23,9,AJ613OLZZUG7V,5,47,0,Rajasthan,Alisha Solid Women's Cycling Shorts,Alisha,5
24,25,9,A3FONPR03H3PJS,5,56,0,Chhattisgarh,Alisha Solid Women's Cycling Shorts,Alisha,5


In [34]:
# product_name = panda_data[df['ProductId'] == '']['ProductName'].values[0]


In [33]:
k = 10
reader = Reader(rating_scale=(1, 5))
surprise_data = Dataset.load_from_df(panda_data[['UserId', 'ProductId', 'Mapped_Score']], reader)

In [34]:
trainset, testset = train_test_split(surprise_data, test_size=.30, random_state=42)

In [35]:
start_time = time.time()
svd_model = SVD(n_epochs=20, lr_all=0.005, reg_all=0.2)
svd_model.fit(trainset)
predictions_svd = svd_model.test(testset)
svd_cv = cross_validate(svd_model, surprise_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
svd_df = pd.DataFrame.from_dict(svd_cv)
svd_described = svd_df.describe()
cv_results = pd.DataFrame([['SVD', svd_described['test_rmse']['mean'], svd_described['test_mae']['mean'],
                           svd_described['fit_time']['mean'], svd_described['test_time']['mean']]],
                            columns = ['Model', 'RMSE', 'MAE', 'Fit Time', 'Test Time'])


# get RMSE
print("\n\n==================== Model Evaluation ===============================")
accuracy.rmse(predictions_svd, verbose=True)
print("=====================================================================")
computational_time = time.time() - start_time
print('\n Computational Time : %0.3fs' %(computational_time))
cv_results

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9939  0.9880  1.0282  0.9922  0.9930  0.9991  0.0147  
MAE (testset)     0.7665  0.7649  0.7936  0.7693  0.7662  0.7721  0.0109  
Fit time          0.36    0.42    0.60    0.39    0.44    0.44    0.08    
Test time         0.06    0.05    0.05    0.06    0.09    0.06    0.01    


==================== Model Evaluation ===============================
RMSE: 1.0096

 Computational Time : 3.592s


,Model,RMSE,MAE,Fit Time,Test Time
0,SVD,0.999074,0.772085,0.443071,0.061177


In [36]:
start_time = time.time()
svdpp_model = SVDpp(n_epochs=25, lr_all=0.01, reg_all=0.4)
svdpp_model.fit(trainset)
predictions_svdpp = svdpp_model.test(testset)
svdpp_cv = cross_validate(svdpp_model, surprise_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
svdpp_df = pd.DataFrame.from_dict(svdpp_cv)
svdpp_described = svdpp_df.describe()
svdpp_cv_results = pd.DataFrame([['SVDpp', svdpp_described['test_rmse']['mean'], svdpp_described['test_mae']['mean'],
                           svdpp_described['fit_time']['mean'], svdpp_described['test_time']['mean']]],
                            columns = ['Model', 'RMSE', 'MAE', 'Fit Time', 'Test Time'])

cv_results = cv_results._append(svdpp_cv_results, ignore_index=True)
print("\n\n==================== Model Evaluation ===============================")
accuracy.rmse(predictions_svdpp, verbose=True)
print("=====================================================================")
computational_time = time.time() - start_time
print('\n Computational Time : %0.3fs' %(computational_time))
cv_results

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9486  0.9288  0.9390  0.9260  0.9316  0.9348  0.0082  
MAE (testset)     0.7214  0.7071  0.7104  0.7063  0.7076  0.7106  0.0056  
Fit time          1.55    0.74    0.82    0.79    0.87    0.95    0.30    
Test time         0.15    0.14    0.14    0.14    0.14    0.14    0.00    


==================== Model Evaluation ===============================
RMSE: 0.9395

 Computational Time : 6.879s


,Model,RMSE,MAE,Fit Time,Test Time
0,SVD,0.999074,0.772085,0.443071,0.061177
1,SVDpp,0.934797,0.710553,0.954753,0.141584


In [ ]:
predictions_svd

[Prediction(uid='A3NIDDT7E7JIFW', iid='B006F2NYI2', r_ui=1.0, est=4.548342096304112, details={'was_impossible': False}),
 Prediction(uid='A17GK9E70O7Y9R', iid='B008Z4VAPM', r_ui=3.0, est=3.552137718336177, details={'was_impossible': False}),
 Prediction(uid='AO2MVU9UIR2BM', iid='B00474HDWS', r_ui=1.0, est=3.8716933958702615, details={'was_impossible': False}),
 Prediction(uid='AIJXKCP2XE3MK', iid='B001CU0N7C', r_ui=5.0, est=4.174439955133264, details={'was_impossible': False}),
 Prediction(uid='A1QUXQA6WSVTH1', iid='B000ENUC3S', r_ui=5.0, est=4.391906453113001, details={'was_impossible': False}),
 Prediction(uid='A31BG4GILLHMQP', iid='B001D0KG4Q', r_ui=3.0, est=3.970968462797069, details={'was_impossible': False}),
 Prediction(uid='AP62C8V1SLZSG', iid='B001E5DWW8', r_ui=5.0, est=4.611321198787846, details={'was_impossible': False}),
 Prediction(uid='A2C0ZF6J18T9WV', iid='B001CD1VI4', r_ui=5.0, est=4.216669470734212, details={'was_impossible': False}),
 Prediction(uid='A1TMAVN4CEM8U8', 

In [37]:
top_n = defaultdict(list)
def get_top_n(predictions, n=k):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

top_n = get_top_n(predictions_svd, n=k)
top_n

defaultdict(list,
            {'A1ENZK86F1VE8P': [(9294, 4.527786463330519)],
             'A1R1BFJCMWX0Y3': [(24694, 3.67410517127269),
              (6870, 3.3958299858686267),
              (7942, 3.0699438690017424)],
             'A3CWJ604OXQ4RX': [(24991, 4.468999554388157),
              (4180, 4.404527697032718)],
             'A3PECZX773ME74': [(23236, 4.371182799413928),
              (25123, 4.0776688788482724),
              (5559, 3.9948140108372776),
              (18912, 3.8930368235461392),
              (9341, 3.2980108683977845)],
             'A10RJEQN64ATXU': [(29155, 4.04965765822655),
              (3207, 4.01382187957361),
              (7196, 3.982965389912742),
              (8434, 3.976724130763781)],
             'A3PLWHAOGO8FPR': [(23781, 4.523204577158606),
              (24057, 4.482218579872318),
              (19988, 4.470090462688924)],
             'A3EIS4NH1R2ZKO': [(4822, 4.539865052655084),
              (24935, 4.211475168151173)],
             'A3

In [39]:
import json
formatted_data = {}
for user_id, ratings in top_n.items():
    formatted_ratings = [{"ProductId": product_id, "Rating": rating} for product_id, rating in ratings]
    formatted_data[user_id] = formatted_ratings

# Save the formatted data to a JSON file
with open('formatted_top_n.json', 'w') as json_file:
    json.dump(formatted_data, json_file, indent=4)


In [ ]:
start_time = time.time()
df = pd.read_csv("Reviews.csv", names=["Id","ProductId", "UserId","ProfileName","HelpfulnessNumerator","HelpfulnessDenominator","Score","Time","Summary","Text"])
df.head()
computational_time = time.time() - start_time
print('Done in %0.3fs' %(computational_time))

FileNotFoundError: ignored

In [ ]:
start_time = time.time()
df = pd.read_csv("Reviews.csv", names=["Id","ProductId", "UserId","ProfileName","HelpfulnessNumerator","HelpfulnessDenominator","Score","Time","Summary","Text"])
df.head()
computational_time = time.time() - start_time
print('Done in %0.3fs' %(computational_time))
df1 = df.dropna()
sample_size = 1000  # Adjust as needed
top_k_features = 500  # Adjust as needed

# Sample a subset of your data (assuming df1 is your DataFrame)
df2 = df1.sample(n=sample_size, random_state=42)
df2.index = range(1, len(df2) + 1)
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
df2['Summary'] = df2['Summary'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df2['Summary'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
indices = pd.Series(df2.index, index=df2['ProductId']).drop_duplicates()
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return df2['ProductId'].iloc[movie_indices]
get_recommendations('B001M2BM4I')

KeyError: ignored

In [ ]:
df1 = df.dropna()

In [ ]:
sample_size = 1000  # Adjust as needed
top_k_features = 500  # Adjust as needed

# Sample a subset of your data (assuming df1 is your DataFrame)
df2 = df1.sample(n=sample_size, random_state=42)


In [ ]:
df2

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
548708,548708,B001M2BM4I,A2OH0YZ2XEQA63,Maria Orfan,0,0,4,1309824000,Espresso at home.,"Description said extra, extra bold and it deli..."
300637,300637,B000AY9UIY,A22CW0ZHY3NJH8,Noname,1,1,4,1288224000,A great addition to the Mini-Wheats line,I am a huge fan of Kellogg's Frosted Mini-Whea...
418337,418337,B00267A348,A3F7UDB1JE5V21,M. Russell,2,3,5,1251763200,A different kind of tea,Quite refreshing. If you want something diffe...
550217,550217,B002Z613JO,A3CIAUOZM72H0C,justlikeschleck,1,1,5,1301529600,Great Beans!,I bought a pack of these beans to try what Kon...
500097,500097,B00061MRB8,A2MPBYQKMXHVN1,"Means Johnston III ""Means""",0,0,4,1238025600,Defurr-ums for Tinkerbell,Our Cat Tinkerbell loves cat treats (which cat...
...,...,...,...,...,...,...,...,...,...,...
220717,220717,B000EUG6YA,A1HG5OEMT3QPIP,Michelle P,0,0,5,1340755200,BEST COOKIES EVER!!!,I can't understand how these cookies could pos...
217187,217187,B000ESLJ6C,A2RWPLOHPUX8WB,pateo,0,0,5,1335484800,pero instant natural beverage,"<a href=""http://www.amazon.com/gp/product/B000..."
353984,353984,B0027049E4,A2V3FFFM6MV5OM,"B. Ogatiy ""bogatiy""",0,0,3,1339113600,Not a good substitute for Postum,I ordered this along with Cafix and Roastaroma...
434963,434963,B000FYXBHU,A3JA8EXHWOUV6M,Fish Head Soup,0,0,3,1334793600,Good but could be better.,Overall this brand is good but I think it is l...


In [ ]:
df2.index = range(1, len(df2) + 1)

In [ ]:
df2

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
1,1,B001M2BM4I,A2OH0YZ2XEQA63,Maria Orfan,0,0,4,1309824000,Espresso at home.,"Description said extra, extra bold and it deli..."
2,2,B000AY9UIY,A22CW0ZHY3NJH8,Noname,1,1,4,1288224000,A great addition to the Mini-Wheats line,I am a huge fan of Kellogg's Frosted Mini-Whea...
3,3,B00267A348,A3F7UDB1JE5V21,M. Russell,2,3,5,1251763200,A different kind of tea,Quite refreshing. If you want something diffe...
4,4,B002Z613JO,A3CIAUOZM72H0C,justlikeschleck,1,1,5,1301529600,Great Beans!,I bought a pack of these beans to try what Kon...
5,5,B00061MRB8,A2MPBYQKMXHVN1,"Means Johnston III ""Means""",0,0,4,1238025600,Defurr-ums for Tinkerbell,Our Cat Tinkerbell loves cat treats (which cat...
...,...,...,...,...,...,...,...,...,...,...
996,996,B000EUG6YA,A1HG5OEMT3QPIP,Michelle P,0,0,5,1340755200,BEST COOKIES EVER!!!,I can't understand how these cookies could pos...
997,997,B000ESLJ6C,A2RWPLOHPUX8WB,pateo,0,0,5,1335484800,pero instant natural beverage,"<a href=""http://www.amazon.com/gp/product/B000..."
998,998,B0027049E4,A2V3FFFM6MV5OM,"B. Ogatiy ""bogatiy""",0,0,3,1339113600,Not a good substitute for Postum,I ordered this along with Cafix and Roastaroma...
999,999,B000FYXBHU,A3JA8EXHWOUV6M,Fish Head Soup,0,0,3,1334793600,Good but could be better.,Overall this brand is good but I think it is l...


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
df2['Summary'] = df2['Summary'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df2['Summary'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape


(1000, 1143)

In [ ]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
indices = pd.Series(df2.index, index=df2['ProductId']).drop_duplicates()

In [ ]:
tfidf_matrix

<1000x1143 sparse matrix of type '<class 'numpy.float64'>'
	with 2686 stored elements in Compressed Sparse Row format>

In [ ]:
len(indices)

1000

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return df2['ProductId'].iloc[movie_indices]

In [ ]:
get_recommendations('B001M2BM4I')

514    B001ELL9GI
549    B000UJREM8
260    B000H0ZJHW
14     B001RVFEP2
59     B0013A0QXC
300    B000E1S8NQ
386    B000EMB012
510    B000ETAJZI
867    B000MP1FW0
877    B00375LB80
Name: ProductId, dtype: object